In [25]:
import csv
import pickle as pkl

In [26]:
def read_csv(csv_path):
    content = []

    with open(csv_path, 'r') as csvfile:
        reader = csv.reader(csvfile, quoting = csv.QUOTE_ALL)
        for row in reader: # each row is a list
            content.append(row)

    return content


def list_to_dict(lst):
    dct = {}
    for row in lst:
        dct[row[0]] = {'segment': [float(row[1]), float(row[2])]}
    return dct

In [27]:
csvfile = '/usr/local/data01/zahra/datasets/FACEOFF/tempuckey_ground_truth.csv'
content_list = read_csv(csvfile)
content_json = list_to_dict(content_list)

In [28]:
import json
json.dump(content_json, open('/usr/local/data01/zahra/datasets/FACEOFF/tempuckey_ground_truth.json','w'))